In [59]:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.random as rd

In [60]:
class I:
    S = 0
    E = 1
    I = 2
    R = 3
    N_State = 4
    N_Strata = 300
    
    Inf = 0
    Inc = 1
    Notif = 2
    
    N_Aux = 3

    
class Model:
    def __call__(self, t, y, pars):
        y, aux = y[:-I.N_Aux], y[-I.N_Aux:]
        y = y.reshape(I.N_State, I.N_Strata)
        
        n = y.sum()
        s = y[I.S]
        e = y[I.E]
        i = y[I.I]
        r = y[I.R]
        
        beta, eta, gamma, mu = pars['beta'], pars['eta'], pars['gamma'], pars['mu']

        if t > 380:
            eta *= np.exp(- 0.01 * (t - 380))
        
        foi = beta * i.sum() / n
        inf = foi * s
        act = eta * e
        rec = gamma * i
        die = mu * y
        
        dy, da = np.zeros_like(y), np.zeros_like(aux)
        
        dy[I.S] = die.sum(0) - inf
        dy[I.E] = inf - act
        dy[I.I] = act - rec
        dy[I.R] = rec
        dy -= die
        
        da[I.Inf] = inf.sum()
        da[I.Inc] = act.sum()
        da[I.Notif] = rec.sum()
        
        return np.concatenate([dy.reshape(-1), da.reshape(-1)])
    
    def measure(self, t, y, pars):
        y, aux = y[:-I.N_Aux], y[-I.N_Aux:]
        y = y.reshape(I.N_State, I.N_Strata)
        
        n = y.sum()
        s = y[I.S]
        e = y[I.E]
        i = y[I.I]
        r = y[I.R]
        
        beta, eta, gamma, mu = pars['beta'], pars['eta'], pars['gamma'], pars['mu']

        foi = beta * i.sum() / n
        inf = foi * s
        act = eta * e
        rec = gamma * i
        die = mu * y
        
        return {
            'Time': t,
            'N': n,
            'InfR': inf.sum() / n,
            'IncR': act.sum() / n,
            'CNR': rec.sum() / n,
            'AccInf': aux[I.Inf],
            'ACCInc': aux[I.Inc],
            'AccNotif': aux[I.Notif]
        }

In [61]:
y0 = np.zeros((I.N_State, I.N_Strata))
y0[I.S] = 990
y0[I.I] = 10

y0 = np.concatenate([y0.reshape(-1), np.zeros(I.N_Aux)])
y0


m0 = Model()


pars = {
    'beta': rd.random(I.N_Strata) * 3,
    'eta': 1,
    'gamma': 0.5,
    'mu': 0.05
}


ys = solve_ivp(m0, [0, 400], y0 = y0, args = (pars, ), t_eval=np.linspace(390, 400, 11))

ms = pd.DataFrame([m0.measure(t, ys.y[:, i], pars) for i, t in enumerate(ys.t)])

ms

,Time,N,InfR,IncR,CNR,AccInf,ACCInc,AccNotif
0,390.0,300000.0,0.024458,0.025376,0.021004,3.042933e+06,2.890610e+06,2.619098e+06
1,391.0,300000.0,0.024403,0.025618,0.020930,3.050267e+06,2.897489e+06,2.625392e+06
2,392.0,300000.0,0.024397,0.025795,0.020896,3.057584e+06,2.904367e+06,2.631663e+06
3,393.0,300000.0,0.024374,0.026018,0.020850,3.064899e+06,2.911227e+06,2.637925e+06
4,394.0,300000.0,0.024348,0.026264,0.020801,3.072211e+06,2.918073e+06,2.644175e+06
5,395.0,300000.0,0.024373,0.026438,0.020794,3.079510e+06,2.924924e+06,2.650407e+06
6,396.0,300000.0,0.024334,0.026735,0.020738,3.086821e+06,2.931748e+06,2.656641e+06
7,397.0,300000.0,0.024363,0.026926,0.020737,3.094118e+06,2.938584e+06,2.662856e+06
8,398.0,300000.0,0.024338,0.027221,0.020695,3.101426e+06,2.945398e+06,2.669073e+06
9,399.0,300000.0,0.024358,0.027447,0.020691,3.108726e+06,2.952219e+06,2.675277e+06


In [66]:
ms.AccNotif.diff() / ms.N 

0          NaN
1     0.020980
2     0.020905
3     0.020871
4     0.020835
5     0.020774
6     0.020779
7     0.020717
8     0.020724
9     0.020681
10    0.020677
dtype: float64

In [65]:
(ms.CNR.shift(-1) + ms.CNR) / 2

0     0.020967
1     0.020913
2     0.020873
3     0.020825
4     0.020798
5     0.020766
6     0.020738
7     0.020716
8     0.020693
9     0.020679
10         NaN
Name: CNR, dtype: float64

In [58]:
cnr = np.array(ms.CNR)

(cnr[:-1] + cnr[1:]) / 2

array([0.02362292, 0.02356775, 0.02352211, 0.02348242, 0.02344622,
       0.02342049, 0.02339203, 0.02337548, 0.02335315, 0.02333632])

(11,)